### Домашнее задание №4 к лекции «Продвинутый pandas»

In [1]:
import pandas as pd
import numpy as np

#### Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [2]:
log = pd.read_csv('Основы_pandas/visit_log.csv', sep=';')

In [3]:
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [4]:
%%time
new_column_name = 'source_type' # задаем имя нового столбца
# фильтруем таблицу по разным условиям и заполняем новый столбец соответствующим условием
log.loc[log['traffic_source'].isin(['yandex', 'google']), new_column_name] = 'organic' 
log.loc[(log['traffic_source'].isin(['paid', 'email'])) & (log['region'] == 'Russia'), new_column_name] = 'ad'
log.loc[(log['traffic_source'].isin(['paid', 'email'])) & ~(log['region'] == 'Russia'), new_column_name] = 'other'
# все пустые поля нового столбца - не попавшие под предыдущие условия заполняем traffic_source с сохранением изменений
log[new_column_name].fillna(log['traffic_source'], inplace=True)

CPU times: user 15.5 ms, sys: 1.88 ms, total: 17.4 ms
Wall time: 15.9 ms


In [5]:
log.groupby('source_type', dropna=False).agg({'traffic_source':'unique', 'region':'unique'})

,traffic_source,region
source_type,,
ad,"[paid, email]",[Russia]
direct,[direct],"[Russia, China, India, Belarus, Ukraine, Austr..."
organic,"[yandex, google]","[Russia, Ukraine, Belarus, India, Australia, C..."
other,"[paid, email]","[Belarus, Ukraine, India, China, Australia]"


Все source_type проставились соответствуем условию

#### Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

1. Прочитайте содержимое файла в датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [6]:
urls = pd.read_csv('Основы_pandas/URLs.txt')

In [7]:
urls.shape[0] # проверяем кол-во строк

93

In [8]:
urls.loc[urls['url'].str.contains('/\d{8}\-', regex=True)] # задаем регулярное выражение "/, затем 8 цифр, затем дефис"

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


In [9]:
urls.loc[~urls['url'].str.contains('/\d{8}\-', regex=True)] # для проверки обратное условие предыдущему

,url
0,/world/
1,/latest/
2,/?updated=top
8,/science/
9,/sport/
32,/auto/
34,/scitech/
37,/lifestyle/
39,/latest/?page=1


Как видим из 93 строк первоначального файла 83 в нужным условием, и 9 с прочими url адресами.  
Все верно

#### Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [10]:
ratings = pd.read_csv('ml-latest/ratings.csv')

In [11]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [12]:
grouped_ratings = ratings.groupby('userId').agg({'rating':'count','timestamp':['min','max']})
grouped_ratings.head()

rating           timestamp                    
        count                 min                 max
userId                                               
1          20 2009-12-14 02:51:48 2009-12-14 02:53:25
2          76 1996-06-21 11:09:55 1996-06-21 11:24:06
3          51 2011-02-28 02:53:09 2011-02-28 22:39:47
4         204 2000-02-05 19:25:14 2000-02-08 03:57:54
5         100 2006-11-12 23:10:44 2006-11-12 23:45:45

In [13]:
grouped_ratings.loc[grouped_ratings['rating']['count'] > 100].head()

rating           timestamp                    
        count                 min                 max
userId                                               
4         204 2000-02-05 19:25:14 2000-02-08 03:57:54
8         116 2006-07-31 23:42:20 2006-08-01 23:22:07
15       1700 2001-08-16 04:47:19 2016-07-24 03:25:35
17        363 2005-09-23 09:43:07 2005-09-23 11:57:20
19        423 1997-02-06 00:48:11 1997-02-06 02:16:13

In [14]:
str(grouped_ratings.apply(
    lambda x: x['timestamp']['max'] - x['timestamp']['min'] if x['rating']['count'] > 100 else np.nan, axis=1
).mean())

'463 days 21:28:27.449612400'

Среднее время жизни пользователей, которые выставили более 100 оценок __полтора года__

#### Задание 4
Дана статистика услуг перевозок клиентов компании по типам:  
____________

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов
____________


Необходимо сформировать две таблицы:  
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

In [15]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [16]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [17]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [18]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [19]:
report = pd.concat([rzd, auto, air]).groupby('client_id')[['rzd_revenue', 'auto_revenue', 'air_revenue']].sum()
"""и sum и mean даст одинаковые итоги, но
применяя сумму в кач-ва агрегирующей функции мы сразу избавимся от Nan"""
report

,rzd_revenue,auto_revenue,air_revenue
client_id,,,
111,1093.0,0.0,0.0
112,2810.0,0.0,0.0
113,10283.0,57483.0,0.0
114,5774.0,83.0,0.0
115,981.0,912.0,81.0
116,0.0,4834.0,4.0
117,0.0,98.0,13.0
118,0.0,0.0,173.0


In [20]:
report.join(client_base.set_index('client_id')) # Задаем client_id в базе клиентов как индекс потом применяем join

,rzd_revenue,auto_revenue,air_revenue,address
client_id,,,,
111,1093.0,0.0,0.0,Комсомольская 4
112,2810.0,0.0,0.0,Энтузиастов 8а
113,10283.0,57483.0,0.0,Левобережная 1а
114,5774.0,83.0,0.0,Мира 14
115,981.0,912.0,81.0,ЗЖБИиДК 1
116,0.0,4834.0,4.0,Строителей 18
117,0.0,98.0,13.0,Панфиловская 33
118,0.0,0.0,173.0,Мастеркова 4


Обе таблицы сформированы согласно условию.  
Оставим client_id в индексах